In [11]:
import numpy as np
from enum import IntEnum

# Assign the default scores
match=1
gap_penalty=1
mismatch_penalty=1

# Assign the constant values for the traceback
class Trace(IntEnum): # reference: https://www.geeksforgeeks.org/enum-intenum-in-python/
    STOP = 0
    LEFT = 1 
    UP = 2
    DIAG = 3

# Create a Smith-Waterman local alignment algorithm function with match=1, gap_penalty=1, mismatch_penalty=1 
def align(seq1, seq2):
    row = len(seq1) + 1
    col = len(seq2) + 1
    # Create empty matrice M for storing scores and tracing matrix s for optimal local alignment
    M = np.zeros(shape=(row, col), dtype=int)  
    s = np.zeros(shape=(row, col), dtype=int)  
    
    # Initialize the variables to find the highest score and the location
    max_score = 0
    max_index = (0, 0)
    
    # Calculate the scores for all cells in M
    for i in range(1, row):
        for j in range(1, col):
            # Calculate the diagonal score (match score)
            match_value = match if seq1[i - 1] == seq2[j - 1] else - mismatch_penalty
            diagonal_score = M[i - 1, j - 1] + match_value
            
            # Calculate the up gap score
            up_score = M[i - 1, j] - gap_penalty
            
            # Calculate the left gap score
            left_score = M[i, j - 1] - gap_penalty
            
            # Take the highest score 
            M[i, j] = max(diagonal_score, up_score, left_score, 0)
            
            # Trace where the cell's value is coming from and assign numeric values
            if M[i, j] == 0: 
                s[i, j] = Trace.STOP # assign StOP if the score is 0
            # Assign LEFT if the score is the left_score 
            elif M[i, j] == left_score: 
                s[i, j] = Trace.LEFT
            # Assign UP if the score is the up_score    
            elif M[i, j] == up_score: 
                s[i, j] = Trace.UP
            # Assign DIAG if the score is the diagonal_score   
            elif M[i, j] == diagonal_score: 
                s[i, j] = Trace.DIAG
                
            # Trace the cell with the maximum score
            if M[i, j] >= max_score:
                max_index = (i,j)
                max_score = M[i, j]
    
    # Initialize the variables for tracing the optimal local alignment
    aligned_seq1 = ""
    aligned_seq2 = ""   
    current_aligned_seq1 = ""   
    current_aligned_seq2 = ""  
    max_i = max_index[0]
    max_j = max_index[1]
    
    # Trace the pathway of the local alignment
    while s[max_i, max_j] != Trace.STOP: 
        if s[max_i, max_j] == Trace.DIAG:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = seq2[max_j - 1]
            max_i -=1
            max_j -=1
            
        elif s[max_i, max_j] == Trace.UP:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = '-'
            max_i -=1    
            
        elif s[max_i, max_j] == Trace.LEFT:
            current_aligned_seq1 = '-'
            current_aligned_seq2 = seq2[max_j - 1]
            max_j -=1
            
        aligned_seq1 = aligned_seq1 + current_aligned_seq1
        aligned_seq2 = aligned_seq2 + current_aligned_seq2
    
    # Reverse the order of the aligned sequences
    aligned_seq1 = aligned_seq1[::-1]
    aligned_seq2 = aligned_seq2[::-1]
    
    return aligned_seq1, aligned_seq2, max_score

In [12]:
# Test 1
seq1, seq2, score, M = align('tgcatcgagaccctacgtgac', 'actagacctagcatcgac')
print(seq1)
print(seq2)
print(score)

agacccta-cgt-gac
agacc-tagcatcgac
8


This is one of the possibility: 

agacccta-cgt-gac
agacc-tagcatcgac
8

In [20]:
# Test 2
seq1, seq2, score, M = align('tggtcgagaactacgtgac', 'actagacctaccatcggc')
print(seq1)
print(seq2)
print(score)

agaactac
agacctac
6


There is only one possibility:

agaactac
agacctac
6

In [15]:
# Test 3
seq1, seq2, score, M = align('gtctcgagaactacgtgac', 'agtacacctaccatcggc')
print(seq1)
print(seq2)
print(score)

agaactacgtgac
agtac-acct-ac
5


This is one of the possibility: 
agaactacgtgac
agtac-acct-ac
5

In [16]:
# Assign different gap_penalty score
match=1
mismatch_penalty=1

# Create a Smith-Waterman local alignment algorithm function with match=1, gap_penalty=2, mismatch_penalty=1 
def align(seq1, seq2, gap_penalty=2):

    row = len(seq1) + 1
    col = len(seq2) + 1
    # Create empty matrice M for storing scores and tracing matrix s for optimal local alignment
    M = np.zeros(shape=(row, col), dtype=int)  
    s = np.zeros(shape=(row, col), dtype=int)  
    
    # Initialize the variables to find the highest score and the location
    max_score = 0
    max_index = (0, 0)
    
    # Calculate the scores for all cells in M
    for i in range(1, row):
        for j in range(1, col):
            # calculate the diagonal score (match score)
            match_value = match if seq1[i - 1] == seq2[j - 1] else - mismatch_penalty
            diagonal_score = M[i - 1, j - 1] + match_value
            
            # calculate the up gap score
            up_score = M[i - 1, j] - gap_penalty
            
            # calculate the left gap score
            left_score = M[i, j - 1] - gap_penalty
            
            # take the highest score 
            M[i, j] = max(diagonal_score, up_score, left_score, 0)
            
            # trace where the cell's value is coming from    
            if M[i, j] == 0: 
                s[i, j] = Trace.STOP # assign STOP if the score is 0
            # assign LEFT if the score is the left_score 
            elif M[i, j] == left_score: 
                s[i, j] = Trace.LEFT
            # assign UP if the score is the up_score    
            elif M[i, j] == up_score: 
                s[i, j] = Trace.UP
            # assign DIAG if the score is the diagonal_score   
            elif M[i, j] == diagonal_score: 
                s[i, j] = Trace.DIAG
            # trace the cell with the maximum score
            if M[i, j] >= max_score:
                max_index = (i,j)
                max_score = M[i, j]
    
    # Initialize the variables for tracing the optimal local alignment
    aligned_seq1 = ""
    aligned_seq2 = ""   
    current_aligned_seq1 = ""   
    current_aligned_seq2 = ""  
    max_i = max_index[0]
    max_j = max_index[1]
    
    # Trace the pathway of the local alignment
    while s[max_i, max_j] != Trace.STOP: 
        if s[max_i, max_j] == Trace.DIAG:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = seq2[max_j - 1]
            max_i -=1
            max_j -=1
            
        elif s[max_i, max_j] == Trace.UP:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = '-'
            max_i -=1    
            
        elif s[max_i, max_j] == Trace.LEFT:
            current_aligned_seq1 = '-'
            current_aligned_seq2 = seq2[max_j - 1]
            max_j -=1
            
        aligned_seq1 = aligned_seq1 + current_aligned_seq1
        aligned_seq2 = aligned_seq2 + current_aligned_seq2
    
    # Reverse the order of the aligned sequences
    aligned_seq1 = aligned_seq1[::-1]
    aligned_seq2 = aligned_seq2[::-1]
    
    return aligned_seq1, aligned_seq2, max_score

In [17]:
# Test 1
seq1, seq2, score, M = align('tgcatcgagaccctacgtgac', 'actagacctagcatcgac',gap_penalty=2)
print(seq1)
print(seq2) 
print(score)

gcatcga
gcatcga
7


In [21]:
# Test 2
seq1, seq2, score, M = align('tggtcgagaactacgtgac', 'actagacctaccatcggc', gap_penalty=2)
print(seq1)
print(seq2)
print(score)

agaactac
agacctac
6


In [22]:
# Test 3
seq1, seq2, score, M = align('gtctcgagaactacgtgac', 'agtacacctaccatcggc', gap_penalty=2)
print(seq1)
print(seq2)
print(score)

ctac
ctac
4


In [24]:
# Assign the different scores match = 2, mismatch_penalty = 2, gap_penalty = 3
match=2
mismatch_penalty=2

# Test
seq1, seq2, score, M = align('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', gap_penalty=3)
print(seq1)
print(seq2)
print(score)

gcatcga
gcatcga
14


In [26]:
# Assign the different scores match = 2, mismatch_penalty = 1, gap_penalty = 4
match=2
mismatch_penalty=1

# Test
seq1, seq2, score, M = align('gtctcgagaactacgtgac', 'agtacacctaccatcggc', gap_penalty=4)
print(seq1)
print(seq2)
print(score)

agaactac
acacctac
10
